# 7. Machine translation and advanced recurrent LSTMs and GRUs

* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture9.pdf
* http://www.statmt.org/wmt15/translation-task.html
* https://arxiv.org/pdf/1406.1078.pdf

In [9]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
from copy import deepcopy
import os
flatten = lambda l: [item for sublist in l for item in sublist]

In [10]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [11]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [12]:
def pad_to_batch(batch,x_to_ix,y_to_ix):
    x,y = list(zip(*batch))
    max_x = max([s.size(1) for s in x])
    max_y = max([s.size(1) for s in y])
    x_p,y_p=[],[]
    for i in range(len(batch)):
        if x[i].size(1)<max_x:
            x_p.append(torch.cat([x[i],Variable(LongTensor([x_to_ix['<PAD>']]*(max_x-x[i].size(1)))).view(1,-1)],1))
        else:
            x_p.append(x[i])
        if y[i].size(1)<max_y:
            y_p.append(torch.cat([y[i],Variable(LongTensor([y_to_ix['<PAD>']]*(max_y-y[i].size(1)))).view(1,-1)],1))
        else:
            y_p.append(y[i])
    return torch.cat(x_p),torch.cat(y_p)

In [13]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if w in to_index.keys() else to_index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

### Data load and Preprocessing 

In [14]:
source_corpus = open('../dataset/europarl-v8.fi-en.en','r',encoding='utf-8').readlines()
target_corpus = open('../dataset/europarl-v8.fi-en.fi','r',encoding='utf-8').readlines()

In [15]:
print(len(source_corpus)==len(target_corpus))
print(len(source_corpus))

True
1926114


In [16]:
from nltk.tokenize import TweetTokenizer
tknz = TweetTokenizer()

In [17]:
parallel_corpus = list(zip(source_corpus,target_corpus))
sampled_parallel_corpus = random.sample(parallel_corpus,1000) # we need to sample data for practice
# sampled_parallel_corpus = parallel_corpus

In [18]:
%%time
X_r,y_r=[],[] # raw

for parallel in sampled_parallel_corpus:
    if parallel[1]=='\n':continue
    en,fi = parallel[0][:-1], parallel[1][:-1] # ignore \n
    X_r.append(tknz.tokenize(en))
    y_r.append(tknz.tokenize(fi))

CPU times: user 116 ms, sys: 0 ns, total: 116 ms
Wall time: 114 ms


### Build Vocab

In [19]:
source_vocab = list(set(flatten(X_r)))
target_vocab = list(set(flatten(y_r)))

In [20]:
print(len(source_vocab),len(target_vocab))

4527 7605


In paper, they limit the source and target vocabulary to the most frequent 15,000 words. <br>
There are some problems of softmax operation's complexity to large target. You should determine this hyper-param in practice. 
We'll deal with this large vocab problem later.

In [21]:
VOCAB_LIMIT = 5000 

In [22]:
source_vocab_count = Counter(flatten(X_r))
source_vocab, _ = list(zip(*source_vocab_count.most_common()[:VOCAB_LIMIT]))

target_vocab_count = Counter(flatten(y_r))
target_vocab, _  = list(zip(*target_vocab_count.most_common()[:VOCAB_LIMIT]))
print(len(source_vocab),len(target_vocab))

4527 5000


In [23]:
source2index = {'<PAD>':0,'<UNK>':1,'<SOS>':2,'<EOS>':3}
for vo in source_vocab:
    source2index[vo]=len(source2index)
index2source = {v:k for k,v in source2index.items()}

target2index = {'<PAD>':0,'<UNK>':1,'<SOS>':2,'<EOS>':3}
for vo in target_vocab:
    target2index[vo]=len(target2index)
index2target = {v:k for k,v in target2index.items()}

In [24]:
%%time
X_p,y_p=[],[]

temp = list(zip(X_r,y_r))
random.shuffle(temp)
train_p = temp[:int(len(temp)*0.9)]
test_data = temp[int(len(temp)*0.9):]

for so,ta in train_p:
    X_p.append(prepare_sequence(['<SOS>']+so+['<EOS>'],source2index).view(1,-1))
    y_p.append(prepare_sequence(ta+['<EOS>'],target2index).view(1,-1))
    
train_data = list(zip(X_p,y_p))

CPU times: user 1.35 s, sys: 344 ms, total: 1.69 s
Wall time: 3.86 s


### Modeling 

In [25]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size,hidden_size, n_layers=1):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True,bidirectional=True)
    
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers*2,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
        self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
        self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
    
    def forward(self, inputs, input_masking=None):
        """
        inputs : B,T (LongTensor)
        input_masking : B,T (ByteTensor) if you don't use zero-padding, leave it at that
        """
        hidden = self.init_hidden(inputs)
        
        embedded = self.embedding(inputs)
        output, hidden = self.gru(embedded, hidden)

        real_context=[]
        
        if type(input_masking)==torch.autograd.variable.Variable:
            for i,o in enumerate(output): # B,T,D
                real_length = input_masking[i].data.tolist().count(0) # real length
                real_context.append(o[real_length-1])
            hidden = torch.cat(real_context).view(inputs.size(0),-1).unsqueeze(1)
        else:
            hidden = hidden[-1].unsqueeze(1)
        
        return output, hidden

In [26]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, n_layers=1,dropout_p=0.3):
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        # Define the layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout_p)
        
        self.gru = nn.GRU(embedding_size+hidden_size, hidden_size, n_layers,batch_first=True)
        self.linear = nn.Linear(embedding_size+hidden_size*2, input_size)
    
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
        self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
        self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
        self.linear.weight = nn.init.xavier_uniform(self.linear.weight)
    
    def forward(self, inputs, context,max_length,training=False):
        """
        inputs : B,1 (LongTensor, START SYMBOL)
        context : B,1,H (FloatTensor, Last encoder hidden state)
        max_length : int, max length to decode
        training : bool, this is because adapt dropout only training step.
        """
        # Get the embedding of the current input word
        embedded = self.embedding(inputs)
        hidden = self.init_hidden(inputs)
        if training:
            embedded = self.dropout(embedded)
        
        decode=[]
        # Apply GRU to the output so far
        for i in range(max_length):

            _, hidden = self.gru(torch.cat((embedded,context),2), hidden) # h_t = f(h_{t-1},y_{t-1},c)
            concated = torch.cat((hidden,embedded.transpose(0,1),context.transpose(0,1)),2) # y_t = g(h_t,y_{t-1},c)
            score = self.linear(concated.squeeze(0))
            softmaxed = F.log_softmax(score)
            decode.append(softmaxed)
            decoded = softmaxed.max(1)[1]
            embedded = self.embedding(decoded).unsqueeze(1) # y_{t-1}
            if training:
                embedded = self.dropout(embedded)
            
        #  column-wise concat, reshape!!
        scores = torch.cat(decode,1)
        return scores.view(inputs.size(0)*max_length,-1)

### Train 

It takes for a while

In [27]:
STEP=50
BATCH_SIZE = 16
EMBEDDING_SIZE = 50
HIDDEN_SIZE = 256
LR = 1e-4

In [28]:
encoder = Encoder(len(source2index),EMBEDDING_SIZE,HIDDEN_SIZE,2)
decoder = Decoder(len(target2index),EMBEDDING_SIZE,HIDDEN_SIZE*2)
encoder.init_weight()
decoder.init_weight()

if USE_CUDA:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

loss_function = nn.CrossEntropyLoss(ignore_index=0)
enc_optimizer = optim.Adam(encoder.parameters(),lr=LR)
dec_optimizer = optim.Adam(decoder.parameters(),lr=LR)

In [29]:
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        inputs,targets = pad_to_batch(batch,source2index,target2index)
        
        input_mask = torch.cat([Variable(ByteTensor(tuple(map(lambda s: s ==0, t.data))),volatile=False) for t in inputs]).view(inputs.size(0),-1)
        start_decode = Variable(LongTensor([[target2index['<SOS>']]*targets.size(0)])).transpose(0,1)
        
        encoder.zero_grad()
        decoder.zero_grad()
        output, hidden_c = encoder(inputs,input_mask)
        
        preds = decoder(start_decode,hidden_c,targets.size(1))
                                
        loss = loss_function(preds,targets.view(-1))
        losses.append(loss.data.cpu().numpy()[0] if USE_CUDA else loss.data.numpy()[0] )
        loss.backward()
        torch.nn.utils.clip_grad_norm(encoder.parameters(), 0.5) # gradient clipping
        torch.nn.utils.clip_grad_norm(decoder.parameters(), 0.5) # gradient clipping
        enc_optimizer.step()
        dec_optimizer.step()
        
        if i % 1000==0:
            print("[%d/%d] mean_loss : %0.2f" %(step,STEP,np.mean(losses)))
            losses=[]

[0/50] mean_loss : 8.51
[1/50] mean_loss : 6.45
[2/50] mean_loss : 6.20
[3/50] mean_loss : 6.46
[4/50] mean_loss : 6.35
[5/50] mean_loss : 6.06
[6/50] mean_loss : 5.84
[7/50] mean_loss : 5.94
[8/50] mean_loss : 5.56
[9/50] mean_loss : 5.90
[10/50] mean_loss : 6.20
[11/50] mean_loss : 5.73
[12/50] mean_loss : 5.64
[13/50] mean_loss : 5.69
[14/50] mean_loss : 5.61
[15/50] mean_loss : 5.52
[16/50] mean_loss : 5.47
[17/50] mean_loss : 5.45
[18/50] mean_loss : 5.56
[19/50] mean_loss : 5.62
[20/50] mean_loss : 5.51
[21/50] mean_loss : 5.61
[22/50] mean_loss : 5.58
[23/50] mean_loss : 5.49
[24/50] mean_loss : 5.38
[25/50] mean_loss : 5.48
[26/50] mean_loss : 5.52
[27/50] mean_loss : 5.23
[28/50] mean_loss : 5.42
[29/50] mean_loss : 5.49
[30/50] mean_loss : 5.02
[31/50] mean_loss : 5.20
[32/50] mean_loss : 5.25
[33/50] mean_loss : 5.08
[34/50] mean_loss : 5.16
[35/50] mean_loss : 5.04
[36/50] mean_loss : 5.35
[37/50] mean_loss : 5.24
[38/50] mean_loss : 5.13
[39/50] mean_loss : 4.95
[40/50] me

In [ ]:
torch.save(decoder.state_dict(),os.path.join('../','mt-decoder.pkl'))
torch.save(encoder.state_dict(),os.path.join('../', 'mt-encoder.pkl'))

### Test (Word Error Rate)

In [30]:
accuracy=0

In [31]:
for test in test_data:
    source,target = test[0],test[1]
    source = prepare_sequence(['<SOS>']+source+['<EOS>'],source2index).view(1,-1)
    input_mask = Variable(ByteTensor(tuple(map(lambda s: s ==0, source[0].data)))).view(1,-1)
    start_decode = Variable(LongTensor([[target2index['<SOS>']]*1])).transpose(0,1)
    target = [target2index[t] if t in target2index.keys() else 1 for t in target]
    o, hidden_c = encoder(source,input_mask)
    pred = decoder(start_decode,hidden_c,len(target))
    pred = pred.max(1)[1].data.cpu().tolist() if USE_CUDA else pred.max(1)[1].data.tolist()
    accuracy+=np.equal(pred,target).sum()

_, target_length = list(zip(*test_data))
print(accuracy/len(flatten(target_length)*100))

0.00107742284786
